# Introduction to NLP Fundamentals in Tensorflow

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

2.14.0-dev20230730


In [2]:
import os

if not os.path.exists("helper_functions.py"):
  !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
else:
  print("[INFO] 'helper_functions.py' already exists, skipping download.")

# Import series of helper functions for the notebook (we've created/used these in previous notebooks)
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-08-13 14:24:35--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... 

connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0.001s  

2023-08-13 14:24:36 (12.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



We will be using RNN to train textual information

**Basic Structure of RNN**
* Input layer
* text_vectorizer(input)
* embedding(x)
* LSTM -> activation = tanh
* output_Dense -> activation = sigmoid

### Get the text dataset

In [3]:
if not os.path.exists("nlp_getting_started.zip"):
    !wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
    unzip_data("nlp_getting_started.zip")
else:
  print("[INFO] already exists, skipping download.")

--2023-08-13 14:24:37--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 2404:6800:4007:829::2010, 2404:6800:4007:826::2010, 2404:6800:4007:827::2010, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2404:6800:4007:829::2010|:443... 

connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   189KB/s    in 3.1s    

2023-08-13 14:24:41 (189 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



### Visualizing a text dataset

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [5]:
train_df.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [6]:
#Shuffle to train data
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head(10)

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
5559,7934,rainstorm,NaN,@Calum5SOS you look like you got caught in a r...,0
1765,2538,collision,NaN,my favorite lady came to our volunteer meeting...,1
1817,2611,crashed,NaN,@brianroemmele UX fail of EMV - people want to...,1
6810,9756,tragedy,"Los Angeles, CA",Can't find my ariana grande shirt this is a f...,0
4398,6254,hijacking,"Athens,Greece",The Murderous Story Of AmericaÛªs First Hijac...,1


In [7]:
# test_data
test_df.head(10)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
5,12,NaN,NaN,We're shaking...It's an earthquake
6,21,NaN,NaN,They'd probably still show more life than Arse...
7,22,NaN,NaN,Hey! How are you?
8,27,NaN,NaN,What a nice hat?
9,29,NaN,NaN,Fuck off!


In [8]:
#How many examples in each class
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [9]:
# visualise some random training examples
import random
random_index = random.randint(0,len(train_df)-5)
print(random_index)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
  index,text,target = row
  print(f"Target: {target}, {'(real disaster)' if target > 0 else '(not a real disaster)'}")
  print(f"Text:\n{text}\n")
  print("---")

3870
Target: 1, (real disaster)
Text:
Learning from the Legacy of a Catastrophic Eruption - The New Yorker http://t.co/t344PhNpy9

---
Target: 1, (real disaster)
Text:
We want to see no more Hiroshima and Nagasaki nuclear bomb disaster in this beautiful world.Lets be peaceful &amp; save this human civilization.

---
Target: 0, (not a real disaster)
Text:
On holiday to relax sunbathe and drink ... Putting out bush fires? Not so much ?? #spain https://t.co/dRno7OKM21

---
Target: 0, (not a real disaster)
Text:
If you can't have the roar of the waves a rainstorm &amp; some rollingÛ_ https://t.co/DlVYFvnQee

---
Target: 0, (not a real disaster)
Text:
Ain't no bags in the trunk it's a body

---


In [10]:
len(train_df_shuffled)

7613

In [11]:
#Splitting the data to create validation -> (train + validation)
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [12]:
#Check length
len(train_sentences),len(train_labels),len(val_sentences),len(val_labels)

(6851, 6851, 762, 762)

In [13]:
# Check few sample
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

1. tokenization -> can get too big as the number of text sample increases
2. Embedding -> richer representation of relationship be between tokens, deeper the realtion between words, more the number/embedding for that token

In [14]:
import tensorflow as tf

# Use the default TextVectorization variables
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                                    standardize="lower_and_strip_punctuation", # how to process text
                                                    split="whitespace", # how to split tokens
                                                    ngrams=None, # create groups of n-words?
                                                    output_mode="int", # how to map tokens to numbers
                                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [15]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = round(sum([len(i.split()) for i in train_sentences])/len(train_sentences)) # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=max_vocab_length,
                                                    output_mode="int",
                                                    output_sequence_length=max_length)

In [17]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [18]:
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
@Guy_Reginald lol more than welcome ??????      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,  174,   51,   76, 1569,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [20]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [21]:
tf.random.set_seed(42)

embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

embedding

In [22]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
@kaputt21 Hamburg Police Chief Gregory Wickett has told 7 Eyewitness News he 'can't confirm or deny' an investigation is underway.

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.02615548,  0.03596283, -0.03193288, ..., -0.04215677,
         -0.04517215,  0.02450839],
        [ 0.04794712, -0.00440628,  0.00558112, ...,  0.02908808,
         -0.01055112,  0.03687351],
        [-0.00431956, -0.01643995, -0.00313234, ...,  0.02638216,
         -0.01434306, -0.04094162],
        ...,
        [ 0.00403015,  0.03711834, -0.01238756, ..., -0.03173318,
          0.02159021,  0.03526375],
        [-0.00512243,  0.00519339, -0.02927925, ...,  0.02025126,
         -0.01685586, -0.00014712],
        [ 0.01464624,  0.01270623, -0.04064868, ...,  0.02356646,
          0.03027831,  0.00211463]]], dtype=float32)>

In [23]:
# Check out a single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.02615548,  0.03596283, -0.03193288,  0.02808413, -0.01696178,
        0.00710521, -0.03592349, -0.02334394, -0.00444476, -0.02355561,
        0.03088863, -0.00189409,  0.02129949, -0.00063553,  0.04628089,
        0.04599741, -0.01787822,  0.00503946, -0.01076424, -0.04629968,
       -0.00728178, -0.03432925, -0.03750493,  0.01832289, -0.02760329,
        0.03398905,  0.04847938,  0.00794345, -0.01798973,  0.04390371,
        0.00180838, -0.0391475 , -0.02652582,  0.01523193,  0.02101827,
       -0.01774635, -0.02174931,  0.00253918,  0.04823767, -0.02367487,
        0.04867179,  0.01412921,  0.00623195,  0.03430438,  0.03869802,
        0.01941646,  0.03095542,  0.02672033,  0.01740826,  0.01342591,
       -0.01146766, -0.02638488,  0.0297741 , -0.03014934, -0.03304125,
       -0.00615761, -0.00123803, -0.03858942,  0.02102337,  0.00327063,
        0.01224952,  0.02950187, -0.03490544, -0.00267539,  0.01743373,
        0.037392

### Modelling a text dataset
More specifically, we'll be building the following:

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model
* Model 3: GRU model
* Model 4: Bidirectional-LSTM model
* Model 5: 1D Convolutional Neural Network
* Model 6: TensorFlow Hub Pretrained Feature Extractor
* Model 7: Same as model 6 with 10% of training data



In [24]:
#model 0
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [26]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [27]:
from helper_functions import calculate_results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

#### Model 1: A simple dense model

In [28]:
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"

# Build model with the Functional API
inputs = tf.keras.layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = tf.keras.layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [29]:
# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20230813-142442


Epoch 1/5
215/215 [==============================] - 5s 17ms/step - loss: 0.6092 - accuracy: 0.6920 - val_loss: 0.5359 - val_accuracy: 0.7559
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.4408 - accuracy: 0.8199 - val_loss: 0.4693 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3461 - accuracy: 0.8622 - val_loss: 0.4592 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 1s 3ms/step - loss: 0.2846 - accuracy: 0.8918 - val_loss: 0.4644 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 1s 3ms/step - loss: 0.2377 - accuracy: 0.9121 - val_loss: 0.4770 - val_accuracy: 0.7861


In [30]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 1ms/step - loss: 0.4770 - accuracy: 0.7861


[0.4769628345966339, 0.7860892415046692]

#### Recurrent Neural Network
* RNN's are useful for sequence data
* The premise of a recurrent neural network is to use the representation of previous input to aid the representation of a later input

### Model 2 : LSTM -> Long short term memory

Input(text) -> Toeknize -> Embedding -> Layers(RNN/Dense) -> Output(label probability)

In [31]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)

model_2_embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")


# Create LSTM model
inputs = tf.keras.layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)

print(x.shape)

x = tf.keras.layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
x = tf.keras.layers.LSTM(64)(x) # return vector for whole sequence

print(x.shape)

outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs)
model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_2.summary()

(None, 15, 128)


(None, 64)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 15, 64)            33024     
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                                  

In [32]:
# Fit model
model_2_history = model_2.fit(train_sentences, train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "LSTM")])

Saving TensorBoard log files to: model_logs/LSTM/20230813-142450
Epoch 1/5


215/215 [==============================] - 7s 21ms/step - loss: 0.5099 - accuracy: 0.7466 - val_loss: 0.4628 - val_accuracy: 0.7861
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3252 - accuracy: 0.8701 - val_loss: 0.5121 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2284 - accuracy: 0.9145 - val_loss: 0.5555 - val_accuracy: 0.7598
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1641 - accuracy: 0.9409 - val_loss: 0.6742 - val_accuracy: 0.7835
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1150 - accuracy: 0.9539 - val_loss: 0.8305 - val_accuracy: 0.7625


In [33]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.03900822],
       [0.6284077 ],
       [0.9994671 ],
       [0.13896386],
       [0.00195334],
       [0.9995714 ],
       [0.965141  ],
       [0.99964416],
       [0.9995028 ],
       [0.17703865]], dtype=float32)

In [34]:
# Convert preds to laels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [35]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [36]:
model_2_results = calculate_results(val_labels, model_2_preds)
model_2_results

{'accuracy': 76.24671916010499,
 'precision': 0.7631352147335341,
 'recall': 0.7624671916010499,
 'f1': 0.7607595644846901}

#### Model 3: GRU

1. Gated recurrent unit is effective and popular
2. GRU cell has similar features to LSTM cell but has less paramenter

In [37]:
# Build GRU RNN
inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = tf.keras.layers.GRU(64, return_sequences=True)(x)
# x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
x = tf.keras.layers.GRU(64)(x)
# x = tf.keras.layers.Dense(64,activation="relu")(x)
outputs = tf.keras.layers.Dense(1,activation="sigmoid")(x)

model_3 = tf.keras.Model(inputs,outputs)

model_3.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_3_history = model_3.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences,val_labels))

Epoch 1/5


215/215 [==============================] - 5s 18ms/step - loss: 0.2360 - accuracy: 0.9012 - val_loss: 0.5336 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1602 - accuracy: 0.9421 - val_loss: 0.5937 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1285 - accuracy: 0.9531 - val_loss: 0.7514 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1099 - accuracy: 0.9599 - val_loss: 0.7547 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.0924 - accuracy: 0.9664 - val_loss: 0.7260 - val_accuracy: 0.7887


In [38]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

 1/24 [>.............................] - ETA: 3s

24/24 [==============================] - 0s 1ms/step


array([[0.0600846 ],
       [0.8264787 ],
       [0.9996037 ],
       [0.04250248],
       [0.00101213],
       [0.98864347],
       [0.70060784],
       [0.9997768 ],
       [0.99950564],
       [0.6669447 ]], dtype=float32)

In [39]:
# Convert model 3 pred_probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [40]:
model_3_results = calculate_results(val_labels,model_3_preds)
model_3_results

{'accuracy': 78.87139107611549,
 'precision': 0.7909484312997928,
 'recall': 0.7887139107611548,
 'f1': 0.7867378765635805}

#### Model4 - Bidirectional RNN

In [41]:
# Trying out on my own
text_vectorizer_exp = tf.keras.layers.TextVectorization(max_tokens=10000,
                                      output_sequence_length=15)
text_vectorizer_exp.adapt(train_sentences)

inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer_exp(inputs)
x = tf.keras.layers.Embedding(input_dim=10000,
                              output_dim=128,
                              input_length=15)(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
outputs = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

model_4_exp = tf.keras.Model(inputs,outputs)
model_4_exp.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])
model_4_history_exp = model_4_exp.fit(train_sentences,
                                      train_labels,
                                      validation_data=(val_sentences,val_labels),
                                      epochs=5)

Epoch 1/5
215/215 [==============================] - 7s 22ms/step - loss: 0.5066 - accuracy: 0.7510 - val_loss: 0.4539 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3115 - accuracy: 0.8770 - val_loss: 0.5177 - val_accuracy: 0.7664
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2068 - accuracy: 0.9223 - val_loss: 0.5564 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1411 - accuracy: 0.9545 - val_loss: 0.6678 - val_accuracy: 0.7612
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0993 - accuracy: 0.9664 - val_loss: 0.7840 - val_accuracy: 0.7507


Normal RNN's go from left to right

Bi-directional go from right to left as well as left to right

In [42]:
# model 4
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=10000,
                                                    output_sequence_length=15)
text_vectorizer.adapt(train_sentences)

inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = tf.keras.layers.Embedding(input_dim=10000,
                              output_dim=128,
                              input_length=15)(x)
# x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
outputs = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

model_4 = tf.keras.Model(inputs,outputs)

model_4.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_4_history = model_4.fit(train_sentences,train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels))

Epoch 1/5
215/215 [==============================] - 6s 19ms/step - loss: 0.5063 - accuracy: 0.7497 - val_loss: 0.4580 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.3128 - accuracy: 0.8745 - val_loss: 0.5133 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2145 - accuracy: 0.9186 - val_loss: 0.5587 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1527 - accuracy: 0.9479 - val_loss: 0.6196 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1069 - accuracy: 0.9634 - val_loss: 0.6794 - val_accuracy: 0.7835


In [43]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 0s 1ms/step


array([[0.03953835],
       [0.8144705 ],
       [0.99836904],
       [0.16232441],
       [0.00539128],
       [0.98951423],
       [0.8664646 ],
       [0.9993222 ],
       [0.9992299 ],
       [0.2928263 ]], dtype=float32)

In [44]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [45]:
model_4_results = calculate_results(val_labels,model_4_preds)
model_4_results

{'accuracy': 78.34645669291339,
 'precision': 0.7831762038971263,
 'recall': 0.7834645669291339,
 'f1': 0.7832103815590454}

#### 1D CNN

In [46]:
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = tf.keras.layers.Conv1D(32,
                                 5,
                                 activation="relu",
                                 padding="valid")
conv_1d_output = conv_1d(embedding_test)

max_pool = tf.keras.layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [47]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.05263913, -0.06186633, -0.02827875, ..., -0.0233285 ,
         -0.01306801, -0.00136552],
        [-0.06288031, -0.0706658 , -0.01494047, ...,  0.03592439,
          0.03072703, -0.02100245],
        [-0.0558992 ,  0.01345649, -0.03325709, ...,  0.03489795,
         -0.02885491, -0.03427256],
        ...,
        [-0.00209794, -0.00416578, -0.05000888, ...,  0.02023026,
         -0.0032082 ,  0.04568345],
        [-0.00209794, -0.00416578, -0.05000888, ...,  0.02023026,
         -0.0032082 ,  0.04568345],
        [-0.00209794, -0.00416578, -0.05000888, ...,  0.02023026,
         -0.0032082 ,  0.04568345]]], dtype=float32)>

In [49]:
conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[1.80814900e-02, 3.63051780e-02, 0.00000000e+00, 5.31983078e-02,
         3.78285088e-02, 0.00000000e+00, 0.00000000e+00, 7.32593238e-02,
         0.00000000e+00, 0.00000000e+00, 1.66051202e-02, 5.78369796e-02,
         0.00000000e+00, 9.51729491e-02, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.40495056e-02,
         0.00000000e+00, 7.57319704e-02, 7.61710294e-03, 3.38865966e-02,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.17426009e-03],
        [0.00000000e+00, 3.95292677e-02, 0.00000000e+00, 3.58461961e-02,
         0.00000000e+00, 0.00000000e+00, 2.01272499e-02, 3.19474936e-03,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.65492511e-02,
         1.08705781e-01, 0.00000000e+00, 0.00000000e+00, 2.11508628e-02,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+0

In [50]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.01808149, 0.09535617, 0.0087798 , 0.05319831, 0.03782851,
        0.05658273, 0.02012725, 0.07325932, 0.02425144, 0.03913816,
        0.05743523, 0.05783698, 0.10870578, 0.09517295, 0.00750219,
        0.02115086, 0.01809432, 0.03227729, 0.        , 0.03172997,
        0.        , 0.07537664, 0.00283834, 0.08968439, 0.00295543,
        0.07573197, 0.05046241, 0.0338866 , 0.        , 0.0635473 ,
        0.        , 0.02694501]], dtype=float32)>

In [48]:
# trying on my own
# text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=10000,
#                                                     output_sequence_length=15)
# text_vectorizer.adapt(train_sentences)

# inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
# x = text_vectorizer(inputs)
# x = tf.keras.layers.Embedding(input_dim=10000,
#                               output_dim=128,
#                               input_length=15)(x)
# x = tf.keras.layers.Conv1D(64,3)(x)
# x = tf.keras.layers.GlobalMaxPool1D()(x)
# x = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

# model_5_exp = tf.keras.Model(inputs,outputs)
# model_5_exp.summary()